## **Load libraries and data**

In [83]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [84]:
import pandas as pd
import pythainlp
import gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [85]:
from pythainlp import sent_tokenize, word_tokenize
from pythainlp import  Tokenizer
from pythainlp.util import dict_trie
from pythainlp.corpus.common import thai_words

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
%cd /content/drive/MyDrive/Colab_Notebooks/NIDA_CRM/HW11/

/content/drive/MyDrive/Colab_Notebooks/NIDA_CRM/HW11


In [88]:
df = pd.read_csv('Wongnai Reviews - Small.csv')

In [89]:
df.head()

,Review ID,Review
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...
3,4,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...


Tokenize Words

In [90]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [111]:
  words = ['ทรูคอฟฟี่','ศูนย์บริการ',',เอ๊กซ์เพรสโซ่','มัคคิอาโต้','ซีลอน','ออส่วน','เสริ์ฟ','ปลากระพงทอดน้ำปลา','หอยลายผัดพริกเผา','สลัดกุ้งกรอบ','ข้าวเหนียวมะม่วง','ปิ่นเกล้า','ข้าวผัดกุ้ง','ไม่มี','ปุ๊ป','ชานม','เบค่อน']
  custom_words_list = set(thai_words())
  ## add multiple words
  custom_words_list.update(words)
  ## add word
  # custom_words_list.add('เป็งปุ๊ด')
  # custom_words_list.add('เพ็ญพุธ')
  trie = dict_trie(dict_source=custom_words_list)

In [112]:
def custom_token(sentence):
  merged = ''
  custom_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')
  words = custom_tokenizer.word_tokenize(str(sentence))
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [93]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [113]:
df['Custom_tokenized'] = df['Review'].apply(lambda x: custom_token(x))

In [114]:
df.head(10)

,Review ID,Review,Review_tokenized,Custom_tokenized,topics,score
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...,"เป็น,คน,ที่,ชอบ,ทาน, ,Macchiato, ,เป็นประจำ, ,...","เป็น,คน,ที่,ชอบ,ทาน, ,Macchiato, ,เป็นประจำ, ,...",8,0.994649
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...,"Art, ,of, ,Coffee, ,Kasetsart, ,เป็น,ร้านกาแฟ,...","Art, ,of, ,Coffee, ,Kasetsart, ,เป็น,ร้านกาแฟ,...",8,0.995411
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...,"กวง,ทะเล,เผา, ,อาหารทะเล,เค้า,สด,จริงๆ,เนื้อ,ป...","กวง,ทะเล,เผา, ,อาหารทะเล,เค้า,สด,จริงๆ,เนื้อ,ป...",8,0.743067
3,4,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...,"วันนี้,มีโอกาส,ตื่น,เช้า,ครับ,เลย,ถึง,โอกาส,ออ...","วันนี้,มีโอกาส,ตื่น,เช้า,ครับ,เลย,ถึง,โอกาส,ออ...",8,0.509861
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...,"ชอบ,มา,ทาน,ร้าน,นี้,ถ้า,อยาก,กิน,อาหาร,เวียดนา...","ชอบ,มา,ทาน,ร้าน,นี้,ถ้า,อยาก,กิน,อาหาร,เวียดนา...",8,0.807771
5,6,เป็นร้านที่สะดุดตาที่สุดบนถนนจรัญ เลยก็ว่าได้ ...,"เป็น,ร้าน,ที่,สะดุดตา,ที่สุด,บน,ถนน,จรัญ, ,เลย...","เป็น,ร้าน,ที่,สะดุดตา,ที่สุด,บน,ถนน,จรัญ, ,เลย...",8,0.996395
6,7,สวัสดีเพื่อนๆสมาชิกนักรีวิวร้านอาหารชาว Wongna...,"สวัสดี,เพื่อน,ๆ,สมาชิก,นัก,รีวิว,ร้านอาหาร,ชาว...","สวัสดี,เพื่อน,ๆ,สมาชิก,นัก,รีวิว,ร้านอาหาร,ชาว...",8,0.549240
7,8,Starbucks \nเกี่ยวกับร้าน: \nร้าน Starbucks นี...,"Starbucks, ,\n,เกี่ยวกับ,ร้าน,:, ,\n,ร้าน, ,St...","Starbucks, ,\n,เกี่ยวกับ,ร้าน,:, ,\n,ร้าน, ,St...",8,0.998964
8,9,\nร้านเบอเกอรี่ร้านนี้อยู่ในร้านล้างรถชื่อว่า ...,"\n,ร้าน,เบอ,เก,อ,รี่,ร้าน,นี้,อยู่,ใน,ร้าน,ล้า...","\n,ร้าน,เบอ,เก,อ,รี่,ร้าน,นี้,อยู่,ใน,ร้าน,ล้า...",8,0.997890
9,10,ร้านส้มตำเจ๊พิมอยู่ถนนเลี่ยงตลาดหนองมน\nเป็นร้...,"ร้าน,ส้มตำ,เจ๊,พิม,อยู่,ถนน,เลี่ยง,ตลาด,หนอง,ม...","ร้าน,ส้มตำ,เจ๊,พิม,อยู่,ถนน,เลี่ยง,ตลาด,หนอง,ม...",8,0.668508


## **Create Dictionary**

In [115]:
documents = df['Custom_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [116]:
print(dictionary.token2id.keys())

dict_keys([' ', '20', 'Macchiato', 'กว่า', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'ก็', 'ของ', 'ขาด', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'ที่', 'นึง', 'น้อย', 'บาท', 'ปริมาณ', 'พอ', 'มากกว่า', 'มี', 'มีความสุข', 'ยัง', 'ยิ่ง', 'ละ', 'ลา', 'วัน', 'สั่ง', 'หนัก', 'หลังจากนี้', 'หาความ', 'อยู่', 'อีก', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็น', 'เป็นประจำ', 'เฟล', 'เลย', 'เห็น', 'แก้ว', 'แรก', 'แล้ว', 'โบราณ', 'ได้', 'ไม่', 'ๆ', '\n', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'กับ', 'คาราเมล', 'ชา', 'ซ', 'ซีลอน', 'ดี', 'ต้อง', 'ทำให้', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'บ่อยๆ', 'พนักงาน', 'พูดจา', 'มัคคิอาโต้', 'มาก', 'รสชาติ', 'ราคา', 'ร้าน', 'ร้านกาแฟ', 'ลาเต้', 'ลูกค้า', 'สบาย', 'อยาก', 'อัธยาศัย', 'อัสสัม', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เพราะ', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'ไป', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กัน', 'การ', 'กินน้ำ', 'กุ้ง', 'ก่อน', 'ก้น', 'ขอ', 'ขึ้น', 'ข้าวผัด', 'ข้าวเหนียวมะม่วง', 'คงจะ', 'คว

In [117]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]

In [118]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [100]:
#!pip install --upgrade pandas

In [119]:
num_topics = 20
chunksize = 1000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 20.4 s, sys: 12.1 s, total: 32.5 s
Wall time: 19.2 s


In [48]:
#!cat /usr/local/lib/python3.7/dist-packages/pandas/core/computation/check.py

In [120]:
gensimvis.prepare(model, gensim_corpus, dictionary,mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8      -10.878068   10.538728       1        1  71.040359
14     -60.884026  -75.591820       2        1  10.622657
3       -2.336119  -53.289074       3        1   7.122768
16     -61.385319  -12.914634       4        1   5.141579
5      -28.737743  126.296997       5        1   1.845295
18       1.066606 -123.552811       6        1   1.465362
12      45.140720   -7.947869       7        1   1.299725
1     -140.795975  -98.530548       8        1   1.253198
6       61.531681  -83.240562       9        1   0.074941
15     -72.563057 -140.396881      10        1   0.052878
0       15.984491   63.578186      11        1   0.028441
13    -160.851624   68.945145      12        1   0.006348
17      85.961830   51.922935      13        1   0.006019
19     -99.020836  109.936348      14        1   0.005997
4     -176.764130   -8.681684      15        1   0.005874
2     -117.189880  -37.420200      16        1   0.005837
11     108.829918  -22.946203      17        1   0.005816
7       47.588097  121.934814      18        1   0.005720
9     -106.467834   30.176889      19        1   0.005614
10     -50.613804   59.328842      20        1   0.005572, topic_info=      Term          Freq         Total Category  logprob  loglift
0           11968.000000  11968.000000  Default  30.0000  30.0000
53      \n   2610.000000   2610.000000  Default  29.0000  29.0000
78    ร้าน   1195.000000   1195.000000  Default  28.0000  28.0000
17     ที่   1035.000000   1035.000000  Default  27.0000  27.0000
51     ไม่    961.000000    961.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
716  สไตล์      0.003610     44.030551  Topic20  -7.1145   0.3862
48    แล้ว      0.010326    349.646483  Topic20  -6.0634  -0.6348
274    แต่      0.013074    601.804760  Topic20  -5.8274  -0.9418
143   ด้วย      0.009503    347.913425  Topic20  -6.1465  -0.7129
199    ว่า      0.009910    382.908863  Topic20  -6.1046  -0.7668

[1448 rows x 6 columns], token_table=      Topic      Freq Term
term                      
53        1  0.721264   \n
53        2  0.036772   \n
53        3  0.119891   \n
53        4  0.075842   \n
53        5  0.025664   \n
...     ...       ...  ...
52        7  0.005236    ๆ
52        8  0.006284    ๆ
294       1  0.540856   ๆๆ
294       2  0.360570   ๆๆ
294       8  0.060095   ๆๆ

[1281 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 15, 4, 17, 6, 19, 13, 2, 7, 16, 1, 14, 18, 20, 5, 3, 12, 8, 10, 11])

In [121]:
model.show_topic(1)

[(' ', 0.07904387),
 ('ร้าน', 0.01327933),
 ('มี', 0.012522349),
 ('ก็', 0.011801129),
 ('ที่', 0.010718603),
 ('ให้', 0.009023001),
 ('มา', 0.008980532),
 ('จะ', 0.008652089),
 ('นี้', 0.007691739),
 ('อยู่', 0.007537989)]

In [122]:
df['topics'] = df['Custom_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Custom_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [125]:
df.tail()

,Review ID,Review,Review_tokenized,Custom_tokenized,topics,score
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,นี้,คุณ,เพื่อน,อยาก,ส้มตำ, ,หมู,เฮา,เลย,พา...","ค่ำ,นี้,คุณ,เพื่อน,อยาก,ส้มตำ, ,หมู,เฮา,เลย,พา...",3,0.282767
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"ร้าน,สะอาด,ดี, ,ตกแต่ง,สวยงาม, ,มี,ที่จอดรถ, ,...","ร้าน,สะอาด,ดี, ,ตกแต่ง,สวยงาม, ,มี,ที่จอดรถ, ,...",8,0.926311
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,ๆ, ,รีบ,ๆ, ,วิ่ง,มา,เข่า,ห้องเรียน,แทบ,ไม...","เช้า,ๆ, ,รีบ,ๆ, ,วิ่ง,มา,เข่า,ห้องเรียน,แทบ,ไม...",8,0.138708
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้าน,นี้,เป็น,ร้านกาแฟ,เล็ก,ๆ, ,ข้างๆ, ,ร้าน,ๆ...","ร้าน,นี้,เป็น,ร้านกาแฟ,เล็ก,ๆ, ,ข้างๆ, ,ร้าน,ๆ...",8,0.999080
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,อยู่,ใน,ศูนย์,บริการ,ข...","ทรูคอฟฟี่,สาขา,ซีคอน,อยู่,ใน,ศูนย์บริการ,ของ,ท...",18,0.989046


In [110]:
df.to_csv("wongnai.csv",encoding='utf-8-sig')